# 행정동 별 인구 데이터

In [2]:
import pandas as pd

lp = pd.read_csv("./datasets/LOCAL_PEOPLE_DONG_202501.csv", index_col=False)
code = pd.read_excel("./datasets/행정동코드_매핑정보_20241218.xlsx", index_col=False)

In [3]:
lp_ = (
    lp.loc[(lp["기준일ID"] == lp["기준일ID"].max()) & (lp["시간대구분"] == 0)]
    .reset_index()
    .iloc[:, 1:]
    .copy()
)
code_ = code.loc[1:, ["행자부행정동코드", "행정동명"]]
lp_["행정동코드"] = lp_["행정동코드"].astype(int)
code_["행자부행정동코드"] = code_["행자부행정동코드"].astype(int)

In [4]:
merged_df = pd.merge(
    lp_, code_, left_on="행정동코드", right_on="행자부행정동코드", how="left"
)
merged_df = merged_df.loc[:, [merged_df.columns[-1]] + merged_df.columns[3:-2].tolist()]

# 학교 기본 정보

In [ ]:
# school = pd.read_csv("./datasets/서울시 학교 기본정보.csv", encoding="euc-kr")
# school_ = school[
#     [
#         "학교종류명",
#         "설립구분",
#         "남녀공학구분명",
#         "고등학교구분명",
#         "산업체특별학급존재여부",
#         "고등학교일반실업구분명",
#         "특수목적고등학교계열명",
#         "주야구분명",
#         "주야과정",
#         "계열명",
#         "도로명주소",
#     ]
# ]

In [ ]:
# import requests

# key = "key" # do not share

# url = "https://dapi.kakao.com/v2/local/search/address.json"
# headers = {"Authorization": f"KakaoAK {key}"}

# regions = []

# for idx, address in enumerate(school_["도로명주소"]):
#     params = {"query": address}
#     response = requests.get(url, headers=headers, params=params)

#     try:
#         region = response.json()['documents'][0]['address']['region_3depth_h_name']
#         regions.append(region)
#     except:
#         print(f"Error at {idx}: {address}")
#         regions.append(None)

In [ ]:
# school_.loc[:, "행정동명"] = regions
# school_.to_csv("./datasets/서울시 학교 기본정보_address.csv", encoding="euc-kr", index=False)

In [ ]:
school_ = pd.read_csv(
    "./dataset/서울시 학교 기본정보_address.csv", encoding="euc-kr", index_col=False
)

In [ ]:
school_clean = school_.dropna(subset=["행정동명"])

In [26]:
school_counts = (
    school_clean.groupby("행정동명")["학교종류명"].value_counts().unstack(fill_value=0)
)

if set(["초등학교", "중학교", "고등학교"]).issubset(school_counts.columns):
    school_counts = school_counts[["초등학교", "중학교", "고등학교"]]

school_counts.columns = ["초등학교_수", "중학교_수", "고등학교_수"]

In [31]:
merged_df = pd.merge(
    merged_df, school_counts, left_on="행정동명", right_on="행정동명", how="left"
)

# 지하철/버스 역별 승하차 정보